## Prerequisites

In [1]:
# !pip install googlemaps folium
# !pip install python-dotenv

In [2]:
from dotenv import find_dotenv,load_dotenv
load_dotenv(find_dotenv())

True

In [3]:
import os,json
from datetime import datetime
import numpy as np
import folium
from branca.element import Figure
import googlemaps
gmaps = googlemaps.Client(key=os.environ['GOOGLE_MAPS_API_KEY'])

## Get data from address

In [4]:
# Geocoding an address
address = '1600 Amphitheatre Parkway, Mountain View, CA'
geocode_result = gmaps.geocode(address)
print(json.dumps(geocode_result,indent=2))

[
  {
    "address_components": [
      {
        "long_name": "Google Building 40",
        "short_name": "Google Building 40",
        "types": [
          "premise"
        ]
      },
      {
        "long_name": "1600",
        "short_name": "1600",
        "types": [
          "street_number"
        ]
      },
      {
        "long_name": "Amphitheatre Parkway",
        "short_name": "Amphitheatre Pkwy",
        "types": [
          "route"
        ]
      },
      {
        "long_name": "Mountain View",
        "short_name": "Mountain View",
        "types": [
          "locality",
          "political"
        ]
      },
      {
        "long_name": "Santa Clara County",
        "short_name": "Santa Clara County",
        "types": [
          "administrative_area_level_2",
          "political"
        ]
      },
      {
        "long_name": "California",
        "short_name": "CA",
        "types": [
          "administrative_area_level_1",
          "political"
        ]
    

In [5]:
print(geocode_result[0]['geometry']['location'])

{'lat': 37.4223878, 'lng': -122.0841877}


In [6]:
loc = geocode_result[0]['geometry']['location']
map = folium.Map(location=[loc['lat'],loc['lng']], zoom_start=14)
folium.Marker(location=[loc['lat'],loc['lng']], popup=address).add_to(map)
fig = Figure(width=800, height=600)
fig.add_child(map)

## Get Address from lat and lng

In [23]:
# Look up an address with reverse geocoding
reverse_geocode_result = gmaps.reverse_geocode((40.714224, -73.961452))
print(json.dumps(reverse_geocode_result,indent=2))

[
  {
    "address_components": [
      {
        "long_name": "277",
        "short_name": "277",
        "types": [
          "street_number"
        ]
      },
      {
        "long_name": "Bedford Avenue",
        "short_name": "Bedford Ave",
        "types": [
          "route"
        ]
      },
      {
        "long_name": "Williamsburg",
        "short_name": "Williamsburg",
        "types": [
          "neighborhood",
          "political"
        ]
      },
      {
        "long_name": "Brooklyn",
        "short_name": "Brooklyn",
        "types": [
          "political",
          "sublocality",
          "sublocality_level_1"
        ]
      },
      {
        "long_name": "Kings County",
        "short_name": "Kings County",
        "types": [
          "administrative_area_level_2",
          "political"
        ]
      },
      {
        "long_name": "New York",
        "short_name": "NY",
        "types": [
          "administrative_area_level_1",
          "political"


In [20]:
print(len(reverse_geocode_result))

13


In [22]:
locs = [
    [res['geometry']['location']['lat'],res['geometry']['location']['lng']]
    for res in reverse_geocode_result
]
map = folium.Map(location=list(np.mean(locs,axis=0)), zoom_start=5)
for loc,res in zip(locs,reverse_geocode_result):
    # print(loc)
    folium.Marker(location=loc, popup=res['formatted_address']).add_to(map)
fig = Figure(width=800, height=600)
fig.add_child(map)

## Get directions via public transport

In [10]:
# Request directions via public transit
now = datetime.now()
directions_result = gmaps.directions(
    "Sydney Town Hall",
    "Parramatta, NSW",
    mode="transit",
    departure_time=now
)
print(json.dumps(directions_result,indent=2))

[
  {
    "bounds": {
      "northeast": {
        "lat": -33.8148186,
        "lng": 151.208821
      },
      "southwest": {
        "lat": -33.89778769999999,
        "lng": 151.0017629
      }
    },
    "copyrights": "Map data \u00a92023 Google",
    "legs": [
      {
        "arrival_time": {
          "text": "10:08\u202fAM",
          "time_zone": "Australia/Sydney",
          "value": 1690934900
        },
        "departure_time": {
          "text": "9:30\u202fAM",
          "time_zone": "Australia/Sydney",
          "value": 1690932653
        },
        "distance": {
          "text": "25.1 km",
          "value": 25054
        },
        "duration": {
          "text": "37 mins",
          "value": 2247
        },
        "end_address": "Parramatta NSW 2150, Australia",
        "end_location": {
          "lat": -33.8148186,
          "lng": 151.0017629
        },
        "start_address": "483 George St, Sydney NSW 2000, Australia",
        "start_location": {
          "

In [11]:
res = directions_result[0]
print('distance:',res['legs'][0]['distance']['text'])
print('duration:',res['legs'][0]['duration']['text'])
print('start:',res['legs'][0]['start_location'])
print('end:',res['legs'][0]['end_location'])

distance: 25.1 km
duration: 37 mins
start: {'lat': -33.8731812, 'lng': 151.2070056}
end: {'lat': -33.8148186, 'lng': 151.0017629}


In [12]:
# https://qiita.com/shin1007/items/8f31188b9ac83be9e738
def decode_polyline(enc: str):
    """
    Parameters
    ----------
    enc : str
        encoded string of polyline, which can be aquired via Google Maps API.

    Returns
    -------
    result : list
        each element in `result` contains pair of latitude and longitude.
    """
    if enc == None or enc == '':
        return [[0, 0]]

    result = []
    polyline_chars = list(enc.encode())
    current_latitude = 0
    current_longitude = 0
    try:
        index = 0
        while index < len(polyline_chars):
            # calculate next latitude
            total = 0
            shifter = 0

            while True:
                next5bits = int(polyline_chars[index]) - 63
                index += 1
                total |= (next5bits & 31) << shifter
                shifter += 5
                if not(next5bits >= 32 and index < len(polyline_chars)):
                    break

            if (index >= len(polyline_chars)):
                break

            if((total & 1) == 1):
                current_latitude += ~(total >> 1)
            else:
                current_latitude += (total >> 1)

            # calculate next longitude
            total = 0
            shifter = 0
            while True:
                next5bits = int(polyline_chars[index]) - 63
                index += 1
                total |= (next5bits & 31) << shifter
                shifter += 5
                if not(next5bits >= 32 and index < len(polyline_chars)):
                    break

            if (index >= len(polyline_chars) and next >= 32):
                break

            if((total & 1) == 1):
                current_longitude += ~(total >> 1)
            else:
                current_longitude += (total >> 1)

            # add to return value
            pair = [current_latitude / 100000, current_longitude / 100000]
            result.append(pair)

    except:
        pass
    return result

In [13]:
color_map = {
    'WALKING': '#0000ff',
    'TRANSIT': '#ff0000',
}

In [14]:
steps = []
for i,s in enumerate(directions_result[0]['legs'][0]['steps']):
    print(i,s['travel_mode'])
    steps.append({
        'polyline': decode_polyline(s['polyline']['points']),
        'travel_mode': s['travel_mode'],
        'duration': s['duration']['text'],
        'duration_sec': s['duration']['value']
    })
start,end = res['legs'][0]['start_location'],res['legs'][0]['end_location']
lat,lng = (start['lat']+end['lat'])/2,(start['lng']+end['lng'])/2
map = folium.Map(location=[lat,lng], zoom_start=12)
folium.Marker(location=[start['lat'],start['lng']], popup=res['legs'][0]['start_address']).add_to(map)
folium.Marker(location=[end['lat'],end['lng']], popup=res['legs'][0]['end_address']).add_to(map)

for step in steps:
    folium.vector_layers.PolyLine(
        locations=step['polyline'], popup=f'{step["travel_mode"]} ({step["duration"]})', color=color_map[step['travel_mode']]
    ).add_to(map)
fig = Figure(width=800, height=600)
fig.add_child(map)

0 WALKING
1 TRANSIT
2 WALKING
